# Solving the Traveling Salesman Problem with Covalent and Fixstars Amplify

This sample code uses Covalent and Fixstars Amplify to solve the traveling salesman problem (TSP).

The code is based on a modified version of [Amplify Examples](https://github.com/fixstars/amplify-examples/blob/main/notebooks/ja/examples/tsp.ipynb). Amplify Examples is open source software under the [MIT License](https://github.com/fixstars/amplify-examples/blob/main/LICENSE).

## Environment Setup

### Create SSH Tunnel

To connect to the ABCI system via SSH, create an SSH tunnel or `~/.ssh/config` to be able to login using ProxyJump with reference to https://docs.abci.ai/ja/getting-started/#general-method.
The sample applications assume that you have created an SSH tunnel using port 10022 on localhost by the following command:

```console
ssh -i /path/identity_file -L 10022:es:22 -l username as.abci.ai
```

### Steps to create a virtual environment on the ABCI system

Login to the ABCI system and create a virtual environment with the necessary packages installed to perform the tasks in the sample applications.
Please refer to [Dependent Packages](#dependencies) for the required packages.
For more information on creating a virtual environment on the ABCI system, please refer to https://docs.abci.ai/ja/python/.

Below are the steps to create a virtual environment on the ABCI system named `amplify_env`.

1. Login to the ABCI system using your own account.
2. Run `module load python/3.10/3.10.10` to make python available.
3. Create a virtual environment by running `python3 -m venv amplify_env`. The `amplify_env` directory is also created at this time.

<a id="dependencies"></a>
### Dependent Packages

Install the packages required to run the application in your local machine environment and in the virtual environment on the ABCI system.  
Each sample application requires the packages listed below.  
Bolded packages are required for both the local machine and the virtual environment on the ABCI system, and the rest are required only for the local machine.

The sample application uses covalent-gridengine-plugin.  
Please refer to the README of covalent-gridengine-plugin for information on how to install and use covalent-gridengine-plugin.

- **covalent**
- **amplify<=0.12.1**
- **numpy**
- matplotlib
- covalent-gridengine-plugin

In [1]:
import covalent

Create a GridEngineExecutor object to execute tasks on the ABCI system.

In [2]:
abci_executor = covalent.executor.GridEngineExecutor(
    username="username",      # Enter your ABCI username.
    address="localhost",
    port=10022,
    ssh_key_file="~/.ssh/id_rsa",  # Enter the path to your ssh key file.
    remote_workdir="$HOME/amplify_env",
    poll_freq=30,
    cleanup=True,
    embedded_qsub_args={
        "l": ["rt_F=1", "h_rt=1:00:00"],
    },  # qsub options to be embedded in the script
    qsub_args={
        "g": "groupname",    # Enter your ABCI groupname.
    },  # qsub options to be given when it is run on the command line
    prerun_commands=[
        "source /etc/profile.d/modules.sh",
        "module load python/3.10/3.10.10",
        "source ~/amplify_env/bin/activate",
    ],
    postrun_commands=[],
    bashrc_path="~/.bashrc",
    log_stdout="stdout.log",
    log_stderr="stderr.log",
)

First, we prepare the functions needed to solve the traveling salesman problem.

The following is based on the code contained in [Amplify Examples](https://github.com/fixstars/amplify-examples/blob/main/notebooks/ja/examples/tsp.ipynb), and uses the covalent electron.

However, Note that the sample code below is implemented with amplify v0.12 and is not guaranteed to work with amplify v1.0.0 or later.

In [3]:
import numpy as np

# This task is executed on the local machine.
@covalent.electron
def gen_random_tsp(ncity: int) -> tuple[np.ndarray, np.ndarray]:
    # coordinates
    locations = np.random.uniform(size=(ncity, 2))

    # distance matrix
    all_diffs = np.expand_dims(locations, axis=1) - np.expand_dims(locations, axis=0)
    distances = np.sqrt(np.sum(all_diffs**2, axis=-1))

    return locations, distances

In [4]:
import amplify

# This task is executed on ABCI.
@covalent.electron(executor=abci_executor)
def solve_tsp(ncity: int, distances: np.ndarray) -> np.ndarray:
    gen = amplify.BinarySymbolGenerator()
    q = gen.array(ncity, ncity)

    cost = amplify.einsum("ij,ni,nj->", distances, q, q.roll(-1, axis=0))

    # Constraints on row direction
    row_constraints = [amplify.constraint.one_hot(q[n]) for n in range(ncity)]

    # Constraints on column direction
    col_constraints = [amplify.constraint.one_hot(q[:, i]) for i in range(ncity)]

    constraints = sum(row_constraints) + sum(col_constraints)

    constraints *= np.amax(distances)  # Set the weight of constraints
    model = cost + constraints

    client = amplify.client.FixstarsClient()
    client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Enter your token of Amplify AE.
    client.parameters.timeout = 5000  # timeout in milliseconds

    solver = amplify.Solver(client)

    result = solver.solve(model)
    if len(result) == 0:
        raise RuntimeError("Any one of constraints is not satisfied.")

    energy, values = result[0].energy, result[0].values
    q_values = q.decode(values)
    route = np.where(np.array(q_values) == 1)[1]

    return route

In [5]:
import matplotlib.pyplot as plt

# Function to visualize the obtained TSP solution.
def show_route(route: list, distances: np.ndarray, locations: np.ndarray) -> np.float64:
    ncity = len(route)
    path_length = sum(
        [distances[route[i]][route[(i + 1) % ncity]] for i in range(ncity)]
    )

    x = [i[0] for i in locations]
    y = [i[1] for i in locations]
    plt.figure(figsize=(7, 7))
    plt.title(f"path length: {path_length}")
    plt.xlabel("x")
    plt.ylabel("y")

    for i in range(ncity):
        r = route[i]
        n = route[(i + 1) % ncity]
        plt.plot([x[r], x[n]], [y[r], y[n]], "b-")
    plt.plot(x, y, "ro")
    plt.show()

    return path_length


Combine the above two `electron` to create a `lattice` workflow.

In [ ]:
@covalent.lattice
def workflow(ncity: int) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    locations, distances = gen_random_tsp(ncity)
    route = solve_tsp(ncity, distances)

    return route, distances, locations

Next, start the covalent server with the following command

```console
covalent start
````

With the default configuration, you can view the Covalent GUI by accessing http://localhost:48008/ from your browser.

Finally, execute the workflow. First, dispatch the workflow.

In [ ]:
dispatch_id = covalent.dispatch(workflow)(10)

In [ ]:
print(dispatch_id)

In [ ]:
result = covalent.get_result(dispatch_id, wait=True)
print(result)

Visualize the obtained solution.

In [ ]:
route, distances, locations = result.result

path_length = show_route(route, distances, locations)